## Run Anima on Google Colab via ComfyUI
* 🤗 [Anima Model On Hugging Face](https://huggingface.co/circlestone-labs/Anima)
* 📄 **License**: Provided under the [circlestone-labs-non-commercial-license
](https://huggingface.co/circlestone-labs/Anima/blob/main/LICENSE.md)

In [ ]:
#@title Install ComfyUI and Download Model
!apt -q install -y aria2
%cd /content/
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip -q install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git || true
%cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
!pip install -r requirements.txt



!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/diffusion_models/anima-preview.safetensors \
-d /content/ComfyUI/models/diffusion_models -o anima-preview.safetensors > /dev/null 2>&1 \
&& echo "✅ Diffusion model download complete" || echo "❌ Diffusion model download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/pachiiahri/anima-fp8-comfyui/resolve/main/anima-preview_tcfp8_mixed.safetensors \
-d /content/ComfyUI/models/diffusion_models -o anima-preview_tcfp8_mixed.safetensors > /dev/null 2>&1 \
&& echo "✅ fp8 Diffusion model download complete" || echo "❌ fp8 Diffusion model download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/text_encoders/qwen_3_06b_base.safetensors \
-d /content/ComfyUI/models/text_encoders -o qwen_3_06b_base.safetensors > /dev/null 2>&1 \
&& echo "✅ Text encoder download complete" || echo "❌ Text encoder download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/vae/qwen_image_vae.safetensors \
-d /content/ComfyUI/models/vae -o qwen_image_vae.safetensors > /dev/null 2>&1 \
&& echo "✅ VAE download complete" || echo "❌ VAE download failed"
import requests, json, os

url = "https://raw.githubusercontent.com/NeuralFalconYT/comfyUI_Colab/refs/heads/main/workflow/anima.json"

save_path = "/content/ComfyUI/user/default/workflows"
os.makedirs(save_path, exist_ok=True)

data = requests.get(url).json()

with open(f"{save_path}/anima.json", "w") as f:
    json.dump(data, f, indent=2)

print("✅ Workflow set as default")

from IPython.display import clear_output
clear_output()
from IPython.display import Audio,display
display(Audio("https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a", autoplay=True))

In [ ]:

#@title Run Comfy UI
%cd /content
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true

import subprocess, re,time ,socket,threading

def tunnel_printer(port):
    while True:
        time.sleep(0.5)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            if s.connect_ex(("127.0.0.1", port)) == 0:
                break
        finally:
            s.close()
    print("\nComfyUI is up. Launching cloudflared tunnel...")
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    for line in p.stderr:
        if "trycloudflare.com" in line:
            url = re.findall(r"https?://[^\s]+trycloudflare\.com[^\s]*", line)
            if url:
                print("\n🔗 Access ComfyUI:", url[0])

%cd /content/ComfyUI
PORT=8188
threading.Thread(target=tunnel_printer, args=(PORT,), daemon=True).start()

print("\nStarting ComfyUI...")
!python main.py --listen 0.0.0.0 --port {PORT} --dont-print-server


In [ ]:
#@title Run Gradio UI [Run 2 times first Error second Run]
%cd /content/ComfyUI
import os, sys, importlib

# ---- Fix utils shadowing issue ----
if "utils" in sys.modules:
    del sys.modules["utils"]

importlib.invalidate_caches()

curr_dir=os.getcwd()
sys.path.insert(0,curr_dir)


# ==========================================================
# Imports
# ==========================================================
import random
import numpy as np
from PIL import Image
import torch


# ==========================================================
# Helper function
# ==========================================================
def get_value_at_index(obj, index):
    try:
        return obj[index]
    except KeyError:
        return obj["result"][index]


# ==========================================================
# Initialize ComfyUI backend properly
# ==========================================================
def import_custom_nodes():
    import asyncio
    import execution
    import server
    from nodes import init_extra_nodes

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    prompt_server = server.PromptServer(loop)
    execution.PromptQueue(prompt_server)

    init_extra_nodes()


# Import node mapping AFTER path fixes
from nodes import NODE_CLASS_MAPPINGS


# ==========================================================
# MODEL CACHE (LOAD ONCE)
# ==========================================================
MODEL_CACHE = {}


def load_models_once():
    if MODEL_CACHE:
        return MODEL_CACHE

    print("🚀 Loading ComfyUI models (first time only)...")

    import_custom_nodes()

    # CLIP text encoder
    cliploader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
    clip = cliploader.load_clip(
        clip_name="qwen_3_06b_base.safetensors",
        type="stable_diffusion",
        device="default",
    )

    # VAE
    vaeloader = NODE_CLASS_MAPPINGS["VAELoader"]()
    vae = vaeloader.load_vae("qwen_image_vae.safetensors")

    # Full diffusion model (no quantization)
    unetloader = NODE_CLASS_MAPPINGS["UNETLoader"]()
    model = unetloader.load_unet(
        unet_name="anima-preview.safetensors",
        weight_dtype="default",
    )

    MODEL_CACHE.update({
        "clip": clip,
        "vae": vae,
        "model": model,
    })

    print("✅ Models loaded successfully.")
    return MODEL_CACHE


# ==========================================================
# IMAGE GENERATION FUNCTION
# ==========================================================
OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def generate_image(
    positive_prompt,
    negative_prompt="",
    width=1024,
    height=1024,
    steps=30,
    cfg=4,
):

    models = load_models_once()

    clip = models["clip"]
    vae = models["vae"]
    model = models["model"]

    with torch.inference_mode():

        # Encode prompts
        clipencode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()

        pos = clipencode.encode(
            text=positive_prompt,
            clip=get_value_at_index(clip, 0),
        )

        neg = clipencode.encode(
            text=negative_prompt,
            clip=get_value_at_index(clip, 0),
        )

        # Create latent image
        latentnode = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
        latent = latentnode.generate(
            width=width,
            height=height,
            batch_size=1,
        )

        # Sampling
        sampler = NODE_CLASS_MAPPINGS["KSampler"]()
        samples = sampler.sample(
            seed=random.randint(1, 2**64),
            steps=steps,
            cfg=cfg,
            sampler_name="euler",
            scheduler="simple",
            denoise=1,
            model=get_value_at_index(model, 0),
            positive=get_value_at_index(pos, 0),
            negative=get_value_at_index(neg, 0),
            latent_image=get_value_at_index(latent, 0),
        )

        # Decode image
        decode = NODE_CLASS_MAPPINGS["VAEDecode"]()
        decoded = decode.decode(
            samples=get_value_at_index(samples, 0),
            vae=get_value_at_index(vae, 0),
        )

        # Convert tensor → image
        img_tensor = get_value_at_index(decoded, 0)[0]
        img = Image.fromarray(
            np.clip(img_tensor.cpu().numpy() * 255, 0, 255).astype(np.uint8)
        )

        # Save image
        path = os.path.join(
            OUTPUT_DIR,
            f"anima_{random.randint(1000,9999)}.png"
        )
        img.save(path)

        return path


load_models_once()


# img_path = generate_image(
#     positive_prompt="masterpiece anime girl portrait, cinematic lighting",
#     negative_prompt="low quality, blurry",
# )

# print("🎉 Generated image:", img_path)


import gradio as gr


def ui_generate(pos, neg, w, h, steps, cfg):
    path = generate_image(
        positive_prompt=pos,
        negative_prompt=neg,
        width=w,
        height=h,
        steps=steps,
        cfg=cfg,
    )
    return path

css="""
  .gradio-container
  { font-family: 'SF Pro Display', -apple-system, BlinkMacSystemFont, sans-serif; }
  """
with gr.Blocks(theme=gr.themes.Soft(),css=css) as demo:
    gr.HTML("""
          <div style="text-align: center; margin: 20px auto; max-width: 800px;">
              <h1 style="font-size: 2.5em; margin-bottom: 5px;">Anima</h1>
          </div>""")
    with gr.Row():

        # LEFT SIDE
        with gr.Column(scale=1):
            pos_prompt = gr.Textbox(
                label="Positive Prompt",
                placeholder="Enter positive prompt...",
                lines=4,
            )

            neg_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="Enter negative prompt...",
                lines=2,
            )

            generate_btn = gr.Button("Generate Image", variant="primary")

            # Hidden advanced settings
            with gr.Accordion("Advanced Settings", open=False):
                width = gr.Slider(256, 1536, 1024, step=64, label="Width")
                height = gr.Slider(256, 1536, 1024, step=64, label="Height")
                steps = gr.Slider(10, 60, 30, step=1, label="Steps")
                cfg = gr.Slider(1, 10, 4, step=0.5, label="CFG")

        # RIGHT SIDE
        with gr.Column(scale=1):
            output_img = gr.Image(label="Generated Image")

    generate_btn.click(
        fn=ui_generate,
        inputs=[pos_prompt, neg_prompt, width, height, steps, cfg],
        outputs=output_img,
    )


demo.launch(share=True,debug=True)